In [1]:
import sys
sys.path.insert(0,'/exp/rhuang/meta/icefall/egs/librispeech/ASR/conformer_ctc2_noseg')
sys.path.insert(0,'/exp/rhuang/meta/k2/k2/python')
sys.path.insert(0,'/exp/rhuang/meta/k2/temp.linux-x86_64-cpython-310/lib')
sys.path.insert(0,'/exp/rhuang/meta/icefall')

In [2]:
%cd /exp/rhuang/meta/icefall/egs/librispeech/ASR
%pwd  #look at the current work dir

/exp/rhuang/meta/icefall/egs/librispeech/ASR


'/exp/rhuang/meta/icefall/egs/librispeech/ASR'

In [11]:
import torch
from torch.nn.utils.rnn import pad_sequence

import torchaudio
from IPython.display import Audio

In [4]:
wav_file = "/exp/rhuang/meta/icefall/egs/librispeech/ASR/conformer_ctc2_noseg/scripts/temp/143535.mp3"
waveform, sample_rate = torchaudio.load(wav_file)
if waveform.size(0) > 1:
    # waveform = waveform[:1]
    waveform = torch.mean(waveform, axis=0)
waveform = waveform.squeeze(0)
print(f"waveform: {waveform.shape}, sample_rate: {sample_rate}")
print(f"duratioin: {waveform.shape[0]/sample_rate:.2f} secs")

waveform: torch.Size([51998976]), sample_rate: 44100
duratioin: 1179.12 secs


In [16]:
from data_transforms import *

In [7]:
my_waveforms = []

start = 199948 
end = start + sample_rate * 20
my_waveforms.append(waveform[start: end])

start = 1199948 
end = start + sample_rate * 30
my_waveforms.append(waveform[start: end])

start = 2199948 
end = start + sample_rate * 25
my_waveforms.append(waveform[start: end])

In [9]:
Audio(my_waveforms[0].numpy(), rate=sample_rate)

In [10]:
my_waveforms[0].shape

torch.Size([882000])

In [14]:
lengths = torch.tensor([len(w) for w in my_waveforms])
padded_waveforms = pad_sequence(my_waveforms, batch_first=True, padding_value=0)
print(padded_waveforms.shape)
print(lengths)

torch.Size([3, 1323000])
tensor([ 882000, 1323000, 1102500])


## Speed perturbation

In [21]:
_speed_perturb_transform = torchaudio.transforms.SpeedPerturbation(orig_freq=16000, factors=[0.9, 1.0, 1.1])

In [33]:
new_waveforms, new_lengths = _speed_perturb_transform(
    waveform=padded_waveforms, 
    lengths=lengths,
)
print(new_waveforms.shape)
print(new_lengths)
print(lengths / new_lengths)

# In this way, torchaudio will always perturb the audios in a batch to the same rate, which is not what we want.

torch.Size([3, 1470000])
tensor([ 980000, 1470000, 1225000])
tensor([0.9000, 0.9000, 0.9000])


In [52]:
# Instead we should perturb the audios one by one.

samples = [_speed_perturb_transform(sample.squeeze())[0] for sample in padded_waveforms]
print([len(s) for s in samples])
print([len(s)/l for s, l in zip(samples, lengths)])

[2, 2, 2]
[tensor(0.0007), tensor(0.0006), tensor(0.0007)]


In [57]:
_speed_perturb_transform(padded_waveforms[0].squeeze())

(tensor([-0.1702, -0.1903, -0.1919,  ...,  0.0000,  0.0000,  0.0000]), None)

## Feature extraction

In [34]:
_fbank_transform = LhotseFbank()  # 16kHz

In [42]:
features = _fbank_transform(padded_waveforms, sampling_rate=sample_rate)

In [37]:
padded_waveforms.shape

torch.Size([3, 1323000])

In [43]:
features.shape

torch.Size([3000, 80])

In [51]:
features = [_fbank_transform(sample[0].squeeze(), sampling_rate=sample_rate) for sample in samples]
lengths = torch.tensor([elem.shape[0] for elem in features], dtype=torch.int32)
features = torch.nn.utils.rnn.pad_sequence(features, batch_first=True)
features.shape, lengths

(torch.Size([3, 3333, 80]), tensor([3000, 3333, 3000], dtype=torch.int32))

## SpecAug

In [44]:
_spec_aug_transform = SpecAugment(
    time_warp_factor=80,
    num_frame_masks=10,
    features_mask_size=27,
    num_feature_masks=2,
    frames_mask_size=100,
)

In [ ]:
features = data_pipeline(features)